This script was written by @suriyadeepan and all credit goes to him for the code. Available [here](http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/)

In [2]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.twitter import data
import data_utils

### load data from pickled npy files

In [3]:
metadata, idx_q, idx_a = data.load_data(PATH='/Users/RGD/projects_public_github/StackOverflow-based-chatbot/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [4]:
idx_q.shape

(5532, 1000)

In [5]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [6]:
import seq2seq_wrapper

In [7]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/Users/RGD/projects_public_github/StackOverflow-based-chatbot/seq2seq_wrapper.py'>

In [8]:
metadata['idx2w']

['_',
 'unk',
 'the',
 'to',
 'a',
 'and',
 'you',
 'of',
 'it',
 'in',
 'i',
 'is',
 '',
 'your',
 'can',
 'with',
 'on',
 'how',
 'that',
 'for',
 'or',
 'if',
 'this',
 'be',
 'from',
 'as',
 'use',
 'will',
 'have',
 'are',
 'but',
 'not',
 'my',
 'water',
 'get',
 'do',
 'them',
 'so',
 'out',
 'then',
 'one',
 'up',
 'at',
 'some',
 'an',
 'when',
 'just',
 'make',
 'without',
 'they',
 'its',
 'off',
 'like',
 'way',
 'would',
 'into',
 'also',
 'remove',
 'by',
 'dont',
 'put',
 'more',
 'keep',
 'there',
 'could',
 'using',
 'other',
 'which',
 'work',
 'all',
 'should',
 'any',
 'clean',
 'what',
 'over',
 'try',
 'need',
 'time',
 'may',
 'paper',
 'than',
 'used',
 'very',
 'around',
 'dry',
 'after',
 'small',
 'has',
 'down',
 'much',
 'air',
 'works',
 'back',
 'tape',
 'while',
 'plastic',
 'well',
 'take',
 'best',
 'too',
 'about',
 'no',
 'want',
 'enough',
 'only',
 'might',
 'sure',
 'good',
 'something',
 'find',
 'inside',
 'even',
 'most',
 'before',
 'car',
 'w

#### Length of vocabulary

In [9]:
len(metadata['idx2w'])

6002

### Model defining

In [8]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [ ]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

### Model training

In [ ]:
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>


In [56]:
validX.shape

(46, 20)

In [7]:
#sess = model.restore_last_session()

In [32]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

(256, 20)


In [23]:
input_.shape

(20, 256)

### Results

In [ ]:
replies = []
for ii, oi in zip(input_.T, output):
#     print(ii, oi)
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
#     print(q)
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    print(decoded)
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)